# Comparing Suburbs and Identifying Clusters

In [1]:
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from sklearn.datasets import load_iris
%matplotlib inline

## Data Imports

In [6]:
# 2018-2019 data
df = pd.read_excel(open('data/ts19individual06taxablestatusstateterritorypostcode.xlsx', 'rb'),sheet_name='Table 6A', skiprows=1)

In [7]:
df.head()

,Taxable status,State/ Territory1,Postcode,Number of individuals\nno.,Taxable income or loss3\nno.,Taxable income or loss3\n$,Tax on taxable income\nno.,Tax on taxable income\n$,Medicare levy\nno.,Medicare levy\n$,...,Low income tax offset\n$,Low and middle income tax offset\nno.,Low and middle income tax offset\n$,People with private health insurance\nno.,Private health insurance - your Australian Government rebate received\nno.,Private health insurance - your Australian Government rebate received\n$,Private health insurance rebate\nno.,Private health insurance rebate\n$,Excess private health reduction or refund (rebate reduced)\nno.,Excess private health reduction or refund (rebate reduced)\n$
0,Non Taxable,ACT,2600,1050,1006,12648452,259,610116,13,21049,...,59971,95,22502,742,468,430084,62,34128,53,25386
1,Non Taxable,ACT,2601,1173,1136,9900103,154,176445,3,6188,...,36461,44,8931,377,127,77225,30,10380,7,2297
2,Non Taxable,ACT,2602,4194,4054,52275968,1182,1362902,19,11577,...,265181,404,99011,2175,1294,884597,172,60678,123,42080
3,Non Taxable,ACT,2603,1060,1003,12662531,302,485408,8,9183,...,68818,114,27038,682,418,346810,46,20473,55,26914
4,Non Taxable,ACT,2604,1224,1176,15351889,353,503655,20,16766,...,77744,116,31954,685,448,332876,65,29331,65,18541
